In [1]:
import json
import re

In [2]:
########################### Final Code starting from here #########################################
from nltk.corpus import wordnet
def getSynonyms(token):
    synonyms = []

    for syn in wordnet.synsets(token):
        for l in syn.lemmas():
            synonyms.append(l.name())
    return list(set(synonyms))

In [52]:
getSynonyms("instructor")

['teacher', 'instructor']

In [3]:
def camel_case_paskal_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]
def cleanColName(colName):
    #remove names with one character 

    #seperate pascal and camel cases
    entity= ''
    if colName.find('.')!=-1:
        entity = colName.split('.')
        colName = entity[1]   
        entity = cleanOneToken(entity[0])+'.'
    
    colNames = camel_case_paskal_split(colName)
    
    newColNames=[]
    for name in colNames:
      #lower
      name = name.lower()
      newColNames.append(name)
    
    return newColNames,entity
def cleanOneToken(token):
    token = token.strip()
    token = re.split('_| |-',token)
    # print(token)
    attributes_clean=[]
    entityAlias=''
    for att in token:
      atts,entity = cleanColName(att)
      if entity!='':
        entityAlias=entity
      attributes_clean.extend(atts)
      # print("d",attributes_clean)
    attributes_clean = "_".join(attributes_clean)
    # print(attributes_clean)
    return entityAlias+attributes_clean
def cleanTokens(tokens):
  cleaned_tokens = []
  for token in tokens:
    cleaned_tokens.append(cleanOneToken(token))
  return cleaned_tokens

In [79]:
print(cleanOneToken("member_attendence.member_id")) ############## problem exist here

['member', 'attendence.member', 'id']
d ['member']
['attendence']
d ['attendence']
attendence
d ['member', 'member']
d ['member', 'member', 'id']
member_member_id
attendence.member_member_id


In [4]:
def separate_queries(query):
    queries=[]
    runing_query=[]
    i = 1
    runing_query.append(query[0])
    while i < len(query):
        token = query[i]

        if token == 'select':
            prev_token = query[i-1]
            nested_query = []
            if prev_token == '(':
                #WHILE TO GET BRACKET END
                open_bracket = 1
                while i<len(query) and open_bracket >=1:
                    token = query[i]
                    i+=1
                    if token == '(': open_bracket += 1
                    elif token == ')': open_bracket -= 1
                    if open_bracket == 0: break
                    nested_query.append(token)
            else:
                nested_query = query[i:]
                i = len(query)+1
            nested_queries = separate_queries(nested_query)
            queries.extend(nested_queries)
        if i <= len(query): runing_query.append(token)
        i+=1
    queries.append(runing_query)
    return queries

In [5]:
def getEntitiesAliasesMapping(query):
    entities_aliases = {}
    i = 0
    while i < len(query):
        token = query[i]
        if token == 'as':
            entity = query[i-1]
            alias = query[i+1]
            entities_aliases[alias] = entity
        i+=1
    return entities_aliases

In [21]:
def fixWhereClause(whereClauseInfo):
    if len(whereClauseInfo) == 2:
        whereClauseInfo.append("value")
    if len(whereClauseInfo) == 3 and whereClauseInfo[2] in ["and","or"]:
        tempCondition = whereClauseInfo.pop()
        whereClauseInfo.append("value")
        whereClauseInfo.append(tempCondition)
    if len(whereClauseInfo) == 3:
        whereClauseInfo.append("None")
    return whereClauseInfo

In [19]:
### my getqueries function ###
################# new function #######################
########### changing where clause list #################
stop_words = ["desc","asc"]
AggrFunctions = ["count","sum","min","max","avg"]
tokensToSkip = [",","(",")",".","by","distinct",";"]
endOfWhere = ["and","or","in","not","union","intersect","except","like"]
def get_queries(query,realQuery):
    entities = []
    selectAttrs = []
    joinAttrs = []
    groupByAttrs = []
    orderByAttrs = []
    aggrAttrs = []
    whereAttrs = []
    aliases = getEntitiesAliasesMapping(query)
    i = 0
    token = ""
    wordType = ""
    orderFunction = ""
    aggrType = ""
    whereClauseInfo = []
    while i < len(query):
        token = query[i]
        if token == "select":
            wordType = "select"
            i += 1
            continue
        if token in stop_words:
            i += 1
            continue
        if token == "limit":
            i += 2
            continue
        if (token[0] == "'" or token[0] == '"') and (token[len(token)-1] == "'" or token[len(token)-1] == '"') and len(token) != 1:
            i+=1
            continue
        if token.find('\"') != -1 or token.find("'") != -1 or token.find('"') != -1:
            i += 1
            while i<len(query) and query[i].find('\"') == -1 and query[i].find("'") == -1 and query[i].find('"') == -1:
                i += 1
            i += 1
            continue
        if token in tokensToSkip:
            if token == ")" and wordType == "aggr":
                wordType = "select"
            i += 1
            continue
        if token == 'as':
            i+=2
            continue 
        if token == "group":
            wordType = "groupBy"
            i += 2
            continue
        if token == "order":
            wordType = "orderBy"
            if query[i+2] in AggrFunctions:
                orderFunction = query[i+2]
                i += 3
            else: 
                orderFunction = ""
                i += 2
            continue
        if token in AggrFunctions:
            wordType = "aggr"
            aggrType = token
            i+= 1
            continue
        if token == "from" or token == "join":
            wordType = "entity"
            i += 1
            continue
        if token == "where":
            wordType = "whereClause"
            i += 1
            continue     
        if token == "on":
            wordType = "join"
            i += 1
            continue
        # check for numbers to skip
        if re.match(r'[0-9]+',token):
            i += 1
            continue
        if wordType in ["select","groupBy","orderBy","whereClause","join","aggr"]:
            if i+1 < len(query) and query[i+1] == ".":
                token = aliases.get(token,token)
                token = token + "."+query[i+2]
                i += 2
        if wordType == "whereClause":
            if token not in endOfWhere:
                if token in ["!",">","<"] and query[i+1] == "=":
                    token += query[i+1]
                    i += 1
                whereClauseInfo.append(token)
                i += 1
                continue
            if token == "like":
                whereClauseInfo.append(token)
                i += 1
                continue
            if token == "not" and query[i+1] in ["in","like"]:
                token = token + ' ' + query[i+1]
                whereClauseInfo.append(token)
                i += 2
            ### end of where condition (and , or , in)
            if token in ["and","or","in"]:
                whereClauseInfo.append(token)
                
        if token in ["union","intersect","except"]:
            i += 1
            continue
        if token == "=" and wordType != "whereClause":
            i += 1
            continue
        token = cleanTokens([token])[0]
        sepToken = token.split(".")[-1]
        token_syns = list(set(getSynonyms(sepToken)))
        token_syns = cleanTokens(token_syns)
        synonyms_dict[sepToken] = token_syns
        if wordType == "select":
            selectAttrs.append(token)
        elif wordType == "entity":
            entities.append(token)
        elif wordType == "join":
            joinAttrs.append(token)
        elif wordType == "groupBy":
            groupByAttrs.append(token)
        elif wordType == "orderBy":
            orderByAttrs.append((token, orderFunction))
        elif wordType == "aggr":
            aggrAttrs.append((token,aggrType))
            aggrType = ""
        elif wordType == "whereClause":
            whereClauseInfo = fixWhereClause(whereClauseInfo)
            whereAttrs.append(whereClauseInfo)
            whereClauseInfo = []

        i += 1
    if len(whereClauseInfo) != 0:
        whereClauseInfo = fixWhereClause(whereClauseInfo)
        whereAttrs.append(whereClauseInfo)
    tempDict = {}
    tempDict["query"] = realQuery
    tempDict["entities"] = list(entities)
    tempDict["selectAttrs"] = list(selectAttrs)
    tempDict["joinAttrs"] = list(joinAttrs)
    tempDict["groupByAttrs"] = list(groupByAttrs)
    tempDict["orderByAttrs"] = list(orderByAttrs)
    tempDict["aggrAttrs"] = list(aggrAttrs)
    tempDict["whereAttrs"] = list(whereAttrs)
    return tempDict

In [22]:
synonyms_dict = {}
['select', 'distinct', 'kingdoms', '.', '*', 'from', 'taxonomic_units_types', 'join', 'kingdoms', 'on', 'taxonomic_units_types', '.', 'kingdom_id', '=', 'kingdoms', '.', 'id', 'where', 'taxonomic_units_types', '.', 'rank_name', "!",'=', "'", 'order', "'"]
#test_query = ["select", "name", "from", "track", "where", "milliseconds", "=", "(", "select", "max", "(", "milliseconds", ")", "from", "track", ")"]
#print(test_query)
################ problem ################
#test_query = ["select", "t1", ".", "name", "from", "member", "as", "t1", "join", "member_attendance", "as", "t2", "on", "t1", ".", "member_id", "=", "t2", ".", "member_id", "group", "by", "t2", ".", "member_id", "order", "by", "count", "(", "*", ")", "asc", "limit", "1"]
#################################################
test_query = ['select', 'distinct', 'kingdoms', '.', '*', 'from', 'taxonomic_units_types', 'join', 'kingdoms', 'on', 'taxonomic_units_types', '.', 'kingdom_id', '=', 'kingdoms', '.', 'id', 'where', 'taxonomic_units_types', '.', 'rank_name', '=', "'", 'order', "'"]
nested = separate_queries(test_query)
for query in nested:
    print(get_queries(query,test_query))
    

{'query': ['select', 'distinct', 'kingdoms', '.', '*', 'from', 'taxonomic_units_types', 'join', 'kingdoms', 'on', 'taxonomic_units_types', '.', 'kingdom_id', '=', 'kingdoms', '.', 'id', 'where', 'taxonomic_units_types', '.', 'rank_name', '=', "'", 'order', "'"], 'entities': ['taxonomic_units_types', 'kingdoms'], 'selectAttrs': ['kingdoms.*'], 'joinAttrs': ['types.taxonomic_units_kingdom_id', 'kingdoms.id'], 'groupByAttrs': [], 'orderByAttrs': [], 'aggrAttrs': [], 'whereAttrs': [['taxonomic_units_types.rank_name', '=', 'value', 'None']]}


In [24]:
############# this cell for datasets:  sparc and splash-master and cosql ###########
import json
import re
synonyms_dict = {}
datasets = ["cosql","greatSql","nvBench","sparc","splash","spider","academic","advising","atis","geography","imdb","restaurants","scholar","spider","yelp"]
for ds in datasets:
    keywordsDict = []
    with open('/home/nihal/Desktop/newwhere/GP/notebooks/preparingDatasets/dataset_preprocessing/'+ds+'.json', 'r',encoding='UTF-8') as file:
        data = json.load(file)
        for query in data:
            nestedQueries = []
            query_tokens = query['query_toks']
            separatedQueries = separate_queries(query_tokens)  
            for currentQuery in separatedQueries:
                concatQuery = ' '.join(currentQuery)
                concatQuery = concatQuery.replace(" . ",".")
                tempDict = get_queries(currentQuery,concatQuery)
                nestedQueries.append(tempDict)
            outQuery = {
                'query':query['query'],
                'allQueries':nestedQueries
            }
            keywordsDict.append(outQuery)
    print(len(keywordsDict))
    outputFile = open("./finalOutputs/"+ds+".json", "w")
    json.dump(keywordsDict, outputFile)
    outputFile.close()
outputFile = open("./finalOutputs/synonyms.json", "w")
json.dump(synonyms_dict, outputFile)
outputFile.close()

8919
45942
7247
10228
9493
5183
189
214
1134
259
99
23
284
5183
122


In [22]:
#### splash-master data tokenization #####
import re
import json
Dataset = []
def read_data():
    data = []
    data_path = "/home/nada/GP/GP/GP/notebooks/preparingDatasets/new/Splash-master/data"
    data_files = [data_path + '/' + f for f in os.listdir(data_path) if f.endswith('.json')]
    for data_file in data_files:
        with open(data_file) as f:
            data_object = json.load(f)
            queries = [data['gold_parse'] for data in data_object]
            data.extend(queries)
    return data
data = read_data()

for query in data:
    tempDict = {}
    tempDict["query"] = query
    tokensList = re.split('(\W)', query)
    tokensList = list(filter(None, tokensList))
    tokensList = list(filter(lambda x: x != ' ', tokensList))
    convertedTokensList = (map(lambda x: x.lower(), tokensList))
    tempDict["query_toks"] = list(convertedTokensList)
    Dataset.append(tempDict)

outputFile = open("/home/nada/GP/GP/GP/notebooks/preparingDatasets/dataset_preprocessing/splash.json", "w")
json.dump(Dataset, outputFile)
outputFile.close()

In [23]:
#### sparc data tokenization #####
import re
import json
Dataset = []
def read_data():
    data = []
    data_path = "/home/nada/GP/GP/GP/notebooks/preparingDatasets/new/sparc/sparc"
    data_files = [data_path + '/' + f for f in os.listdir(data_path) if f.endswith('.json')]
    for data_file in data_files:
        with open(data_file) as f:
            data_object = json.load(f)
            interactions = [data['interaction'] for data in data_object]
            for interaction  in interactions:
                for query in interaction:
                    data.append(query['query'])
    return data
data = read_data()
for query in data:
    tempDict = {}
    tempDict["query"] = query
    tokensList = re.split('(\W)', query)
    tokensList = list(filter(None, tokensList))
    tokensList = list(filter(lambda x: x != ' ', tokensList))
    convertedTokensList = (map(lambda x: x.lower(), tokensList))
    tempDict["query_toks"] = list(convertedTokensList)
    Dataset.append(tempDict)

outputFile = open("/home/nada/GP/GP/GP/notebooks/preparingDatasets/dataset_preprocessing/sparc.json", "w")
json.dump(Dataset, outputFile)
outputFile.close()

In [24]:
#### nvBench data tokenization #####
import re
import json
Dataset = []
def read_data():
    data = []
    with open("/home/nada/GP/GP/GP/notebooks/preparingDatasets/new/nvBench.json") as f:
        data_object = json.load(f)
        queries = [data['vis_query']['data_part']['sql_part'] for data in data_object.values()]
        data.extend(queries)
    return data
data = read_data()
for query in data:
    tempDict = {}
    tempDict["query"] = query
    tokensList = re.split('(\W)', query)
    tokensList = list(filter(None, tokensList))
    tokensList = list(filter(lambda x: x != ' ', tokensList))
    convertedTokensList = (map(lambda x: x.lower(), tokensList))
    tempDict["query_toks"] = list(convertedTokensList)
    Dataset.append(tempDict)

outputFile = open("/home/nada/GP/GP/GP/notebooks/preparingDatasets/dataset_preprocessing/nvBench.json", "w")
json.dump(Dataset, outputFile)
outputFile.close()

In [25]:
#### Cosql data tokenization #####
import re
import json
Dataset = []
def read_data():
    data=[]
    data_path = "/home/nada/GP/GP/GP/notebooks/preparingDatasets/new/cosql_dataset/cosql_dataset/system_response_generation"
    data_files = [data_path + '/' + f for f in os.listdir(data_path) if f.endswith('.json')]
    for data_file in data_files:
        with open(data_file) as f:
            data_object = json.load(f)
            queries = [{"query":data['query'],"query_toks":data["query_toks"]}for data in data_object]
            data.extend(queries)
    return data 
data = read_data()
for query in data:
    tempDict = {}
    tempDict["query"] = query["query"]
    tempDict["query_toks"] = query['query_toks']
    Dataset.append(tempDict)

outputFile = open("/home/nada/GP/GP/GP/notebooks/preparingDatasets/dataset_preprocessing/cosql.json", "w")
json.dump(Dataset, outputFile)
outputFile.close()


In [26]:
#### Topics data tokenization #####
import re
import json
import os
datapath ="/home/nada/GP/GP/GP/notebooks/preparingDatasets/new/topics/"
data_files = os.listdir(datapath) 
for data_file in data_files:
    Dataset = []
    def read_data(datapth):
        data=[]
        with open(datapth) as f:
            data_object = json.load(f)
            queries = [data['sql'] for data in data_object]
            for q in queries:
                data.extend(q)
        return data
    data = read_data(datapath+data_file)
    for query in data:
        tempDict = {}
        tempDict["query"] = query
        tokensList = re.split('(\W)', query)
        tokensList = list(filter(None, tokensList))
        tokensList = list(filter(lambda x: x != ' ', tokensList))
        convertedTokensList = (map(lambda x: x.lower(), tokensList))
        tempDict["query_toks"] = list(convertedTokensList)
        Dataset.append(tempDict)

    outputFile = open("/home/nada/GP/GP/GP/notebooks/preparingDatasets/dataset_preprocessing/"+data_file, "w")
    json.dump(Dataset, outputFile)
    outputFile.close()

In [27]:
#### GreatSql data tokenization #####
import re
import json
import os
Dataset = []
def read_data():
    data=[]
    datapath ="/home/nada/GP/GP/GP/notebooks/preparingDatasets/new/GreatSQL-master/data"
    data_files = os.listdir(datapath) 
    for data_file in data_files:
        with open(datapath+"/"+data_file) as f:
            data_object = json.load(f)
            queries = [data['sql'] for data in data_object]
            data.extend(queries)
    return data
data = read_data()
for query in data:
    tempDict = {}
    tempDict["query"] = query
    tokensList = re.split('(\W)', query)
    tokensList = list(filter(None, tokensList))
    tokensList = list(filter(lambda x: x != ' ', tokensList))
    convertedTokensList = (map(lambda x: x.lower(), tokensList))
    tempDict["query_toks"] = list(convertedTokensList)
    Dataset.append(tempDict)

outputFile = open("/home/nada/GP/GP/GP/notebooks/preparingDatasets/dataset_preprocessing/greatSql.json", "w")
json.dump(Dataset, outputFile)
outputFile.close()

In [ ]:
############# old function #######################
### my getqueries function ###
stop_words = ['in','intersect','except','union','not',"desc","asc","or","and","like"]
AggrFunctions = ["count","sum","min","max","avg"]
tokensToSkip = [",","(",")",".","=","!","by","distinct",">",">=","<","<=",";"]
def get_queries(query,realQuery):
    entities = []
    selectAttrs = []
    joinAttrs = []
    groupByAttrs = []
    orderByAttrs = []
    aggrAttrs = []
    whereAttrs = []
    aliases = getEntitiesAliasesMapping(query)
    i = 0
    token = ""
    wordType = ""
    orderFunction = ""
    whereCondition = ""
    while i < len(query):
        token = query[i]
        if token == "select":
            wordType = "select"
            i += 1
            continue
        if token in stop_words:
            i += 1
            continue
        if token == "limit":
            i += 2
            continue
        if token.find('\"') != -1 or token.find("'") != -1 or token.find('"') != -1:
            i += 1
            while i<len(query) and query[i].find('\"') == -1 and query[i].find("'") == -1 and query[i].find('"') == -1:
                i += 1
            i += 1
            continue
        if token.find("'")!=-1 or token.find('"')!=-1:
            i+=1
            continue
        if token in tokensToSkip:
            if token == ")" and wordType == "aggr":
                wordType = "select"
            i += 1
            continue
        if token == 'as':
            i+=2
            continue 
        if token == "group":
            wordType = "groupBy"
            i += 2
            continue
        if token == "order":
            wordType = "orderBy"
            if query[i+2] in AggrFunctions:
                orderFunction = query[i+2]
                i += 3
            else: 
                orderFunction = ""
                i += 2
            continue
        if token in AggrFunctions:
            wordType = "aggr"
            i+= 1
            continue
        if token == "from" or token == "join":
            wordType = "entity"
            i += 1
            continue
        if token == "where":
            wordType = "whereClause"
            i += 1
            continue
        if token == "on":
            wordType = "join"
            i += 1
            continue
        # check for numbers to skip
        if re.match(r'[0-9]+',token):
            i += 1
            continue
        if wordType in ["select","groupBy","orderBy","whereClause","join","aggr"]:
            if i+1 < len(query) and query[i+1] == ".":
                token = aliases.get(token,token)
                token = token + "."+query[i+2]
                i += 2
            if wordType == "whereClause":
                whereConditionsList = [">","<","=","!"]
                if i+1 < len(query):
                    if query[i+1] in whereConditionsList:
                        if i+2 < len(query) and query[i+2] in whereConditionsList:
                            whereCondition = query[i+1] + query[i+2]
                            i += 2
                        else:
                            whereCondition = query[i+1]
                            i += 1
            
                
        token = cleanTokens([token])[0]
        sepToken = token.split(".")[-1]
        token_syns = list(set(getSynonyms(sepToken)))
        token_syns = cleanTokens(token_syns)
        synonyms_dict[sepToken] = token_syns
        if wordType == "select":
            selectAttrs.append(token)
        elif wordType == "entity":
            entities.append(token)
        elif wordType == "join":
            joinAttrs.append(token)
        elif wordType == "groupBy":
            groupByAttrs.append(token)
        elif wordType == "orderBy":
            orderByAttrs.append((token, orderFunction))
        elif wordType == "aggr":
            aggrAttrs.append(token)
        elif wordType == "whereClause":
            whereAttrs.append((token,whereCondition))
            whereCondition = ""
        i += 1
    tempDict = {}
    tempDict["query"] = realQuery
    tempDict["entities"] = list(entities)
    tempDict["selectAttrs"] = list(selectAttrs)
    tempDict["joinAttrs"] = list(joinAttrs)
    tempDict["groupByAttrs"] = list(groupByAttrs)
    tempDict["orderByAttrs"] = list(orderByAttrs)
    tempDict["aggrAttrs"] = list(aggrAttrs)
    tempDict["whereAttrs"] = list(whereAttrs)
    return tempDict